In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima.arima import auto_arima
from scipy import signal
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Visual libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Show all Columns and Rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# %matploblib inline, standarize figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize']= 12,5

### Explorer Dataset

In [ ]:
# Load data set
df = pd.read_csv('teleco_time_series .csv')

In [ ]:
#Create Date from Days
df['Date'] = (pd.date_range(datetime(2020,1,1), periods=df.shape[0]))

#df['Date']= pd.to_datetime(df['Date'],infer_datetime_format=True)
df.set_index('Date', inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

### Revenue in Million over the years

### Part C1

In [ ]:
sns.regplot(x=df.Day, y=df.Revenue, data=df)
plt.xticks(rotation = 'vertical')
plt.title('Time Series: Revenue Over Two Years')
plt.ylabel('($Millions) Revenue')
plt.show()

### Clean Dataset

In [ ]:
#Drop Days column
df.drop(columns=['Day'], inplace=True)

In [ ]:
# Count of missing values per columns
df.isna().sum()

In [ ]:
# Using box plot plot to identify outliers
Revenue = df['Revenue']
Revenue.plot.box()

In [ ]:
# Investigate distribution of Revenue column using histogram
df["Revenue"].plot(kind = "hist", title = 'Revenue Histogram')

In [ ]:
# Create a new column with standarized Income values
df["Revenue_z"] = stats.zscore(df["Revenue"])

In [ ]:
# Based on the z score isolate the outliers
df_revenue_outliers = df.query('Revenue_z > 3 | Revenue_z < -3')

In [ ]:
# Create a new data set for the outliers and sort it in descending order
df_revenue_outliers_sort_values = df_revenue_outliers.sort_values(['Revenue_z'], ascending = False)

In [ ]:
# List out the outliers
df.drop(columns=['Revenue_z'], inplace=True)
df_revenue_outliers_sort_values['Revenue'].head()

### Checking for trend using P-value

In [ ]:
pre_eval = df['Revenue']
pre_eval_result = adfuller(pre_eval, autolag='AIC')
print('ADF Statistic: %f' % pre_eval_result[0])
print('p-value: %f' % pre_eval_result[1])
print('Critical Values:')
for key, value in pre_eval_result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
# Apply the difference to make the data non-stationary
df_stationary = df.Revenue.diff().dropna()

In [ ]:
sns.lineplot(data=df_stationary)
plt.xticks(rotation = 'vertical')
plt.title('Stationarity, No Trends Over Months')
plt.plot()

In [ ]:
df_stationary.head()

### Adfuller after differencing

In [ ]:
_eval = df_stationary
_eval_result = adfuller(_eval, autolag='AIC')
print('ADF Statistic: %f' % _eval_result[0])
print('p-value: %f' % _eval_result[1])

In [ ]:
# Store the new dataset
clean_df = df_stationary.copy()

In [ ]:
# Split the data to 80/20
train_df = clean_df[:576] # 80%
test_df = clean_df[577:] # 20%
print('Train Size: ',train_df.shape)
print('Test Size: ',test_df.shape)

In [ ]:
# Save Data Fram to CSV
clean_df.to_csv('Cleaned_D213_TimeSeriesData.csv')

In [ ]:
fig, ax = plt.subplots()
train_df.plot(ax=ax)
test_df.plot(ax=ax)
plt.xticks(rotation = 'vertical')
plt.ylabel('Price in Mils')
plt.xlabel('Train vs Test Dataset')
plt.show()

### auto_arima

In [ ]:
stepwiseARIMA =  auto_arima(train_df, trace = True,
                          suppress_warnings = True,  # we don't want convergence warnings
                          stepwise = True )

for k,v in stepwiseARIMA.get_params().items():
    if k == 'order' or k == 'seasonal_order':
        print (k,v)

### Spectral Density

In [ ]:
f, Pxx_den = signal.periodogram(train_df)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-5, 1e5])
plt.title('Spectral Density')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Spectral Density')
plt.show()

### ACF/PACF

In [ ]:
# Auto Correlation Function
plot_acf(train_df, lags=20, zero=False)
plt.xlabel('Lags')
plt.show()

In [ ]:
# Partial Auto Correlation Function
plot_pacf(train_df, lags=10, zero=False)
plt.xlabel('Lags')
plt.show()

### Decompose

In [ ]:
results=seasonal_decompose(train_df, period=12)
results.plot().legends

In [ ]:
pd.Series(results.trend).rename('Trend over Days').plot(legend=True)

In [ ]:
pd.Series(results.seasonal).rename('Seasonality').plot(legend=True)

In [ ]:
pd.Series(results.resid).rename('Residuality').plot(legend=True)

### Train ARIMA First Model based on auto_arima

In [ ]:
first_ar_model = sm.tsa.statespace.SARIMAX(train_df, order=(stepwiseARIMA.order))
first_ar_model_fit = first_ar_model.fit()
first_ar_model_fit.summary()

In [ ]:
#First Model prediction
start = len(train_df)
end = len(train_df)+len(test_df)-1

first_ar_model_pred = pd.Series(first_ar_model_fit.predict(start=start, end=end, typ='levels')).rename('Predictions')
first_ar_model_pred.index = df.index[start:end+1]
first_ar_model_pred.head()

In [ ]:
plt.plot(test_df, label='Actual')
plt.plot(first_ar_model_pred,color='r', label='Prediction')
plt.legend()
plt.show()

#### Sarimax - forecast

In [ ]:
second_ar_model = sm.tsa.statespace.SARIMAX(df, order=(stepwiseARIMA.order))
second_ar_model = second_ar_model.fit()
second_ar_model.summary()

In [ ]:
prediction = second_ar_model.get_prediction(start=-90, dynamic = False)
prediction_ci = prediction.conf_int()
prediction_ci.head()

In [ ]:
#Visualize the forecasting
ax = df.plot(label = 'observed')
prediction.predicted_mean.plot(ax = ax, label='Prediction')
ax.fill_between(prediction_ci.index, prediction_ci.iloc[:, 0], prediction_ci.iloc[:, 1], color = 'k', alpha = 0.2)
ax.set_xlabel("Date")
ax.set_ylabel('Revenue (in Millions)')
plt.title('Telcom Revenue Forecast for next 90 days')
plt.legend()
plt.show()

In [ ]:
# Evaluation metrics are Squared Mean Error(SME) and Root Mean Squared Error(RMSE)
y_hat = prediction.predicted_mean
y_truth = df.Revenue

mse = ((y_hat - y_truth) ** 2).mean()
rmse = np.sqrt(mse)
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))